# Set things up

## Import libraries

In [1]:
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(),'../CIBUSmod'))

Import CIBUSmod and packages for handling data and plotting

In [2]:
import CIBUSmod as cm
import CIBUSmod.utils.plot as plot

import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Set up session and define scenarios

In [3]:
# Get years from scenario workbook
years = [x.split('_')[1] for x in pd.read_excel('scenarios/slv_2025.xlsx', sheet_name='DemandAndConversions').columns if 'y_' in x]

# Create session
session = cm.Session(
    name = 'SLV',
    data_path = '../CIBUSmod/data',
    data_path_output = 'output',
    data_path_scenarios = 'scenarios'
)

# Add scenarios
session.add_scenario(
    name = 'SLV',
    scenario_workbooks = 'slv_2025',
    years = years
)

A scenario with the name 'SLV' already exists use .update_scenario() or .remove_scenario() instead.


In [8]:
# Import
from concurrent.futures import ProcessPoolExecutor, as_completed
from runCIBUSmod import do_run

# Create list of scenarios/years to run
runs = [(s,y) for s, y in session.iterate('all')]
runs

[('SLV', '0'),
 ('SLV', '1110'),
 ('SLV', '1120'),
 ('SLV', '1210'),
 ('SLV', '1220'),
 ('SLV', '1310'),
 ('SLV', '1320'),
 ('SLV', '2110'),
 ('SLV', '2120'),
 ('SLV', '2210'),
 ('SLV', '2220'),
 ('SLV', '2310'),
 ('SLV', '2320')]

In [ ]:
%%time
# Do the multi-processing
with ProcessPoolExecutor(max_workers=10) as executor:
    
    futures = {executor.submit(do_run, session, scn_year) : scn_year for scn_year in runs}

    for future in as_completed(futures):
    
        scn, year = futures[future]
           
        try:
            t = future.result()
        except Exception as ee:
            print(f'(!!!) {scn}, {year} failed with the exception: {ee}')
        else:
            m = int(t/60)
            s = int(round(t - m*60))
            print(f'{scn}, {year} finished successfully in {m}min {s}s')
                    